# IndianPremierLeague Auction Analysis

In [96]:
#Install pyspark
!pip install pyspark


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [97]:
#import libraries

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.types import *

In [98]:
#Start Spark Session

spark = SparkSession.builder.getOrCreate()

In [99]:
schema = StructType() \
        .add("",IntegerType(),True) \
        .add("Player", StringType(),True) \
        .add("Base Price",StringType(), True) \
        .add("TYPE",StringType(),True) \
        .add("COST IN ₹ (CR.)",FloatType(),True) \
        .add("Cost IN $ (000)",FloatType(),True) \
        .add("2021 Squad", StringType(),True) \
        .add("Team", StringType(),True)

## Data Extraction

In [100]:
df = spark.read.csv("cpl_2022_dataset.csv", header=True, schema=schema)

## Data Exploration

In [101]:
df.printSchema()

root
 |-- : integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Base Price: string (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- COST IN ₹ (CR.): float (nullable = true)
 |-- Cost IN $ (000): float (nullable = true)
 |-- 2021 Squad: string (nullable = true)
 |-- Team: string (nullable = true)



In [102]:
#Total number of rows and columns

print('Rows: ', df.count())
print('Columns:', len(df.columns))

Rows:  633
Columns: 8


In [103]:
#Unique Values in 'Base Price' Column

df.select('Base Price').distinct().collect()

[Row(Base Price='20 Lakh'),
 Row(Base Price='2 Cr'),
 Row(Base Price='Draft Pick'),
 Row(Base Price='40 Lakh'),
 Row(Base Price='30 Lakh'),
 Row(Base Price='1 Cr'),
 Row(Base Price='50 Lakh'),
 Row(Base Price='1.5 Cr'),
 Row(Base Price='75 Lakh'),
 Row(Base Price='Retained')]

In [104]:
#Distribution of values inside column 'Base Price'

df.groupBy('Base Price').count().show()

+----------+-----+
|Base Price|count|
+----------+-----+
|   20 Lakh|  344|
|      2 Cr|   48|
|Draft Pick|    6|
|   40 Lakh|   16|
|   30 Lakh|    9|
|      1 Cr|   33|
|   50 Lakh|  104|
|    1.5 Cr|   20|
|   75 Lakh|   26|
|  Retained|   27|
+----------+-----+



In [105]:
df.show(5)

+---+---------------+----------+-----------+---------------+---------------+----------+--------------+
|   |         Player|Base Price|       TYPE|COST IN ₹ (CR.)|Cost IN $ (000)|2021 Squad|          Team|
+---+---------------+----------+-----------+---------------+---------------+----------+--------------+
|  0|    Rashid Khan|Draft Pick|     BOWLER|           15.0|         1950.0|       SRH|Gujarat Titans|
|  1|  Hardik Pandya|Draft Pick|ALL-ROUNDER|           15.0|         1950.0|        MI|Gujarat Titans|
|  2|Lockie Ferguson|      2 Cr|     BOWLER|           10.0|         1300.0|       KKR|Gujarat Titans|
|  3|  Rahul Tewatia|   40 Lakh|ALL-ROUNDER|            9.0|         1170.0|        RR|Gujarat Titans|
|  4|   Shubman Gill|Draft Pick|     BATTER|            8.0|         1040.0|       KKR|Gujarat Titans|
+---+---------------+----------+-----------+---------------+---------------+----------+--------------+
only showing top 5 rows



# Data Transformation
## Data cleaning (Handling Missing Values, Renaming Columns)

In [106]:
#Rename Column

df2 = df.withColumnRenamed("","SlNo")\
      .withColumnRenamed("Player","player")\
      .withColumnRenamed("Base Price","base_price")\
      .withColumnRenamed("Type","type") \
      .withColumnRenamed("COST IN ₹ (CR.)","cost_inr") \
      .withColumnRenamed("Cost IN $ (000)","cost_usd") \
      .withColumnRenamed("2021 Squad","twentyOne_team") \
      .withColumnRenamed("Team","twentyTwo_team")

In [107]:
df2.show(3)

+----+---------------+----------+-----------+--------+--------+--------------+--------------+
|SlNo|         player|base_price|       type|cost_inr|cost_usd|twentyOne_team|twentyTwo_team|
+----+---------------+----------+-----------+--------+--------+--------------+--------------+
|   0|    Rashid Khan|Draft Pick|     BOWLER|    15.0|  1950.0|           SRH|Gujarat Titans|
|   1|  Hardik Pandya|Draft Pick|ALL-ROUNDER|    15.0|  1950.0|            MI|Gujarat Titans|
|   2|Lockie Ferguson|      2 Cr|     BOWLER|    10.0|  1300.0|           KKR|Gujarat Titans|
+----+---------------+----------+-----------+--------+--------+--------------+--------------+
only showing top 3 rows



## Checking For Null/missing Values

In [108]:
# Checking For Nul Values

#spark.sql("""SELECT * FROM ipl WHERE cost_usd IS NULL""").show()
df2.select("*").where(df2.cost_usd.isNull()).show(5)
df2.select("*").where(df2.twentyOne_team.isNull()).show(5)
df2.select("*").where(df2.player.isNull()).show(5)
df2.select("*").where(df2.base_price.isNull()).show(5)
df2.select("*").where(df2.twentyTwo_team.isNull()).show(5)
df2.select("*").where(df2.type.isNull()).show(5)

+----+---------------+----------+-----------+--------+--------+--------------+--------------+
|SlNo|         player|base_price|       type|cost_inr|cost_usd|twentyOne_team|twentyTwo_team|
+----+---------------+----------+-----------+--------+--------+--------------+--------------+
| 237|   Suresh Raina|      2 Cr|     BATTER|    NULL|    NULL|           CSK|        Unsold|
| 238|    Steve Smith|      2 Cr|     BATTER|    NULL|    NULL|            DC|        Unsold|
| 239|Shakib Al Hasan|      2 Cr|ALL-ROUNDER|    NULL|    NULL|           KKR|        Unsold|
| 240|    Amit Mishra|    1.5 Cr|     BOWLER|    NULL|    NULL|            DC|        Unsold|
| 241|    Adil Rashid|      2 Cr|     BOWLER|    NULL|    NULL|          PBKS|        Unsold|
+----+---------------+----------+-----------+--------+--------+--------------+--------------+
only showing top 5 rows

+----+-------------------+----------+------------+--------+--------+--------------+--------------+
|SlNo|             player|base

## Filling Null/missing Values with appropriate values

In [109]:
df2 = df2.fillna(value=0.0,subset=["cost_inr"]).orderBy('cost_inr')

In [110]:
df2 = df2.fillna(value=0.0,subset=["cost_usd"]).orderBy('cost_usd')

In [111]:
df2 = df2.fillna(value="NotPartOfAnyTeam",subset=["twentyOne_team"]).orderBy('cost_usd')

In [112]:
df2.show(5)

+----+---------------+----------+-----------+--------+--------+--------------+--------------+
|SlNo|         player|base_price|       type|cost_inr|cost_usd|twentyOne_team|twentyTwo_team|
+----+---------------+----------+-----------+--------+--------+--------------+--------------+
| 237|   Suresh Raina|      2 Cr|     BATTER|     0.0|     0.0|           CSK|        Unsold|
| 242|    Imran Tahir|      2 Cr|     BOWLER|     0.0|     0.0|           CSK|        Unsold|
| 238|    Steve Smith|      2 Cr|     BATTER|     0.0|     0.0|            DC|        Unsold|
| 239|Shakib Al Hasan|      2 Cr|ALL-ROUNDER|     0.0|     0.0|           KKR|        Unsold|
| 240|    Amit Mishra|    1.5 Cr|     BOWLER|     0.0|     0.0|            DC|        Unsold|
+----+---------------+----------+-----------+--------+--------+--------------+--------------+
only showing top 5 rows



In [113]:
#Dropping inr Column

df3 = df2.drop('cost_inr')


In [114]:
#Check updated Dataframe

df3.show()

+----+--------------------+----------+------------+--------+----------------+--------------+
|SlNo|              player|base_price|        type|cost_usd|  twentyOne_team|twentyTwo_team|
+----+--------------------+----------+------------+--------+----------------+--------------+
| 237|        Suresh Raina|      2 Cr|      BATTER|     0.0|             CSK|        Unsold|
| 257|       Ishant Sharma|    1.5 Cr|      BOWLER|     0.0|              DC|        Unsold|
| 238|         Steve Smith|      2 Cr|      BATTER|     0.0|              DC|        Unsold|
| 239|     Shakib Al Hasan|      2 Cr| ALL-ROUNDER|     0.0|             KKR|        Unsold|
| 240|         Amit Mishra|    1.5 Cr|      BOWLER|     0.0|              DC|        Unsold|
| 241|         Adil Rashid|      2 Cr|      BOWLER|     0.0|            PBKS|        Unsold|
| 242|         Imran Tahir|      2 Cr|      BOWLER|     0.0|             CSK|        Unsold|
| 243|       Mujeeb Zadran|      2 Cr|      BOWLER|     0.0|          

# Data Analysis

## Top 3 batsman who got paid the most

In [115]:
df3.select('player','cost_usd').where(df3.type=='BATTER').orderBy('cost_usd', ascending=False).show(3)

+---------------+--------+
|         player|cost_usd|
+---------------+--------+
|   Rohit Sharma|  2080.0|
|    Virat Kohli|  1950.0|
|Kane Williamson|  1820.0|
+---------------+--------+
only showing top 3 rows



In [116]:
df3.createOrReplaceTempView('ipl')

In [117]:
spark.sql("""SELECT player, cost_usd FROM ipl WHERE type = 'BATTER' ORDER BY 2 DESC LIMIT 3""").show()

+---------------+--------+
|         player|cost_usd|
+---------------+--------+
|   Rohit Sharma|  2080.0|
|    Virat Kohli|  1950.0|
|Kane Williamson|  1820.0|
+---------------+--------+



## Top 5 bowlers who get paid the most

In [118]:
df3.select('player','cost_usd').filter(df3.type=='BOWLER').orderBy('cost_usd',ascending=False).show(5)

+---------------+--------+
|         player|cost_usd|
+---------------+--------+
|    Rashid Khan|  1950.0|
|  Deepak Chahar|  1820.0|
| Jasprit Bumrah|  1560.0|
| Shardul Thakur|  1397.5|
|Lockie Ferguson|  1300.0|
+---------------+--------+
only showing top 5 rows



In [119]:
spark.sql("""SELECT player, cost_usd FROM ipl WHERE type = 'BOWLER' ORDER By 2 DESC LIMIT 5 """).show()

+--------------+--------+
|        player|cost_usd|
+--------------+--------+
|   Rashid Khan|  1950.0|
| Deepak Chahar|  1820.0|
|Jasprit Bumrah|  1560.0|
|Shardul Thakur|  1397.5|
|    Avesh Khan|  1300.0|
+--------------+--------+



## 5 lowest paid wicket-keeper


In [120]:
df3.select('player', 'cost_usd').filter((df3.type=='WICKETKEEPER') & (df3.twentyTwo_team!='Unsold')).orderBy('cost_usd').show(5)

+---------------+--------+
|         player|cost_usd|
+---------------+--------+
|  Jitesh Sharma|    26.0|
|    Aryan Juyal|    26.0|
|  N. Jagadeesan|    26.0|
|Luvnith Sisodia|    26.0|
| Baba Indrajith|    26.0|
+---------------+--------+
only showing top 5 rows



In [121]:
spark.sql("""SELECT player, cost_usd FROM ipl WHERE type = 'WICKETKEEPER' AND twentyTwo_team != 'Unsold' ORDER By 2 LIMIT 5 """).show()

+---------------+--------+
|         player|cost_usd|
+---------------+--------+
|    Dhruv Jurel|    26.0|
|  Jitesh Sharma|    26.0|
|  N. Jagadeesan|    26.0|
|    Aryan Juyal|    26.0|
|Luvnith Sisodia|    26.0|
+---------------+--------+



## Average pay for Batsman, Bowler, All-Rounder, Wicket-Keeper

In [122]:
from pyspark.sql import functions as F
df3.groupBy('type').agg(F.round(F.mean('cost_usd'),2).alias('average_pay')).show()

+------------+-----------+
|        type|average_pay|
+------------+-----------+
|WICKETKEEPER|     278.99|
|      BOWLER|     142.91|
| ALL-ROUNDER|     170.48|
|      BATTER|     214.56|
+------------+-----------+



In [123]:
spark.sql("""SELECT type, round(avg(cost_usd),2) average_pay FROM ipl GROUP BY 1 ORDER BY 2 DESC""").show()

+------------+-----------+
|        type|average_pay|
+------------+-----------+
|WICKETKEEPER|     278.99|
|      BATTER|     214.56|
| ALL-ROUNDER|     170.48|
|      BOWLER|     142.91|
+------------+-----------+



## List of Retained players with team name and salary

In [124]:
df3.select('player', 'twentyOne_team', 'cost_usd').where(df3.base_price == 'Retained').orderBy('cost_usd',ascending=False).show(10)

+---------------+--------------+--------+
|         player|twentyOne_team|cost_usd|
+---------------+--------------+--------+
|Ravindra Jadeja|           CSK|  2080.0|
|   Rishabh Pant|            DC|  2080.0|
|   Rohit Sharma|            MI|  2080.0|
|    Virat Kohli|           RCB|  1950.0|
|   Sanju Samson|            RR|  1820.0|
|Kane Williamson|           SRH|  1820.0|
|       MS Dhoni|           CSK|  1560.0|
|  Andre Russell|           KKR|  1560.0|
| Mayank Agarwal|          PBKS|  1560.0|
| Jasprit Bumrah|            MI|  1560.0|
+---------------+--------------+--------+
only showing top 10 rows



In [125]:
spark.sql("""SELECT player, twentyOne_team, cost_usd FROM ipl WHERE base_price = 'Retained' ORDER By 3 DESC LIMIT 10""").show()

+---------------+--------------+--------+
|         player|twentyOne_team|cost_usd|
+---------------+--------------+--------+
|Ravindra Jadeja|           CSK|  2080.0|
|   Rishabh Pant|            DC|  2080.0|
|   Rohit Sharma|            MI|  2080.0|
|    Virat Kohli|           RCB|  1950.0|
|   Sanju Samson|            RR|  1820.0|
|Kane Williamson|           SRH|  1820.0|
| Jasprit Bumrah|            MI|  1560.0|
|  Andre Russell|           KKR|  1560.0|
| Mayank Agarwal|          PBKS|  1560.0|
|       MS Dhoni|           CSK|  1560.0|
+---------------+--------------+--------+

